In [10]:
!git clone https://github.com/elenadigrigoli/FDS-project.git

fatal: destination path 'FDS-project' already exists and is not an empty directory.


In [11]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Trasformazioni delle immagini
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Ridimensiona le immagini
    transforms.ToTensor(),         # Converti in tensore
])

# Carica il dataset
dataset_path = "/content/FDS-project/data"
train_dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

# Crea un DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Visualizza le classi
print(train_dataset.classes)


['battery', 'biological', 'cardboard', 'clothes', 'glass', 'metal', 'paper', 'plastic', 'shoes', 'trash']


In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader
from torch.cuda import is_available


In [13]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

def create_dataloaders(dataset_path, train_split=0.7, val_split=0.2, test_split=0.1, batch_size=32):
    """
    Crea DataLoader per training, validation e test set.

    Args:
        dataset_path (str): Percorso alla cartella principale del dataset.
        train_split (float): Percentuale del dataset da usare per il training (default: 70%).
        val_split (float): Percentuale del dataset da usare per la validazione (default: 20%).
        test_split (float): Percentuale del dataset da usare per il test (default: 10%).
        batch_size (int): Dimensione del batch per i DataLoader (default: 32).

    Returns:
        tuple: DataLoader per train, validation e test set.
    """
    # Trasformazioni delle immagini
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Ridimensiona le immagini
        transforms.ToTensor(),         # Converte in tensori
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalizza i pixel
    ])

    # Carica l'intero dataset
    full_dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

    # Calcola le dimensioni dei sotto-dataset
    dataset_size = len(full_dataset)
    train_size = int(train_split * dataset_size)
    val_size = int(val_split * dataset_size)
    test_size = dataset_size - train_size - val_size

    # Suddividi il dataset
    train_dataset, val_dataset, test_dataset = random_split(
        full_dataset, [train_size, val_size, test_size]
    )

    # Crea DataLoader per ogni sotto-dataset
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, test_loader


In [14]:
# Percorso alla directory principale del dataset
dataset_path = "/content/FDS-project/data"

# Crea DataLoader per train, validation e test set
train_loader, val_loader, test_loader = create_dataloaders(
    dataset_path=dataset_path,
    train_split=0.7,  # 70% per il training
    val_split=0.2,    # 20% per la validazione
    test_split=0.1,   # 10% per il test
    batch_size=32     # Dimensione del batch
)

# Verifica il numero di campioni
print(f"Train set size: {len(train_loader.dataset)}")
print(f"Validation set size: {len(val_loader.dataset)}")
print(f"Test set size: {len(test_loader.dataset)}")


Train set size: 15199
Validation set size: 4342
Test set size: 2173


In [15]:
# Trasformazioni per il dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Ridimensiona le immagini
    transforms.ToTensor(),         # Converte in tensori
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalizzazione
])

# Carica il dataset
#train_dataset = datasets.ImageFolder(root="/content/FDS-project/train", transform=transform)
#val_dataset = datasets.ImageFolder(root="/content/repository_name/val", transform=transform)

# DataLoader

# Numero di classi
num_classes = len(train_dataset.classes)
print(f"Numero di classi: {num_classes}")


Numero di classi: 10


In [16]:
# Carica ResNet-50 pre-addestrato
model = models.resnet50(pretrained=True)

# Congela i pesi delle feature estrattive
for param in model.parameters():
    param.requires_grad = False

# Modifica il classificatore finale
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Sposta il modello su GPU (se disponibile)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 201MB/s]


In [17]:
criterion = nn.CrossEntropyLoss()  # Funzione di perdita per classificazione
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)  # Ottimizza solo i parametri del classificatore


In [18]:
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=10):
    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")

        # Modalità addestramento
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Statistiche
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_loss = running_loss / len(train_loader)
        train_accuracy = correct / total * 100
        print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%")

        # Modalità validazione
        model.eval()
        running_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)

                outputs = model(images)
                loss = criterion(outputs, labels)

                running_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_loss = running_loss / len(val_loader)
        val_accuracy = correct / total * 100
        print(f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")

    return model


In [19]:
model = train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=10)


Epoch 1/10
Train Loss: 0.4820, Train Accuracy: 85.64%
Val Loss: 0.2961, Val Accuracy: 89.96%
Epoch 2/10
Train Loss: 0.2802, Train Accuracy: 90.73%
Val Loss: 0.2294, Val Accuracy: 92.24%
Epoch 3/10
Train Loss: 0.2471, Train Accuracy: 91.76%
Val Loss: 0.2598, Val Accuracy: 91.34%
Epoch 4/10
Train Loss: 0.2261, Train Accuracy: 92.35%
Val Loss: 0.2236, Val Accuracy: 92.08%
Epoch 5/10
Train Loss: 0.2025, Train Accuracy: 93.14%
Val Loss: 0.2075, Val Accuracy: 92.77%
Epoch 6/10
Train Loss: 0.1965, Train Accuracy: 93.28%
Val Loss: 0.1962, Val Accuracy: 93.57%
Epoch 7/10
Train Loss: 0.1982, Train Accuracy: 93.31%
Val Loss: 0.2033, Val Accuracy: 93.14%
Epoch 8/10
Train Loss: 0.1905, Train Accuracy: 93.50%
Val Loss: 0.2186, Val Accuracy: 93.11%
Epoch 9/10
Train Loss: 0.1790, Train Accuracy: 93.77%
Val Loss: 0.1879, Val Accuracy: 94.06%
Epoch 10/10
Train Loss: 0.1706, Train Accuracy: 94.16%
Val Loss: 0.1985, Val Accuracy: 93.83%


In [20]:
torch.save(model.state_dict(), "model.pth")


In [21]:
# Carica i pesi del modello salvati
model.load_state_dict(torch.load("model.pth"))
model.eval()  # Imposta il modello in modalità valutazione


<ipython-input-21-9ff2e84e713d>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("model.pth"))


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [22]:
# Disattiva il calcolo dei gradienti per velocizzare la valutazione
with torch.no_grad():
    all_preds = []
    all_labels = []

    for images, labels in test_loader:
        images = images.to(device)  # Sposta le immagini su GPU (se disponibile)
        labels = labels.to(device)  # Sposta le etichette su GPU (se disponibile)

        # Passa le immagini al modello
        outputs = model(images)

        # Ottieni le classi con probabilità massima
        _, preds = torch.max(outputs, 1)

        # Salva le previsioni e le etichette vere
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())


In [23]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(all_labels, all_preds)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


Test Accuracy: 94.52%
